<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 4: Aproximación de funciones

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

from matplotlib import pyplot as plt

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func

from anmi.T4 import metodo_ruffini

In [ ]:
x, y, z, a, lambda_ = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda')

El objetivo de la aproximación de funciones es encontrar una función de caracteristicas menores o con términos más fáciles para computar, que se aproxime a una función dada en un intervalo o conjunto de puntos determinado. A nivel computacional, también es más eficiente y estable poder emplear cierto tipo de funciones de aproximación.

## Representación anidada (algoritmo de Horner) / Método de Ruffini
Si tenemos un polinomio $p(x) = a_0 + a_1x + a_2x^2+\cdots+a_nx^n$, entonces podemos hacer el polinomio más compacto, en la forma $p(x) = q_1(q_2(\cdots(q_n(x)))$, tal que $q_i(x) = a_{i-1} + a_i(x)$.

In [ ]:
poli_1 = Poly(x**3 - x**2 + 2*x - 5)

In [ ]:
metodo_ruffini(poli_1, 1)

## Aproximación de funciones
Si $f$ es una función en un espacio vectorial $V$ podemos definir una norma $||\;\;||$, que es una aplicación que cumple:
* $||f|| = 0  \iff f = 0$
* $||\lambda f|| = |\lambda|\,||f||$
* $||f + g|| \le ||f|| + ||g||$

Si fijamos un subespacio $U \subset V$, podemos buscar una función $g\in U$ tal que sea la más similar.

Si queremos medir la distancia entre 2 funciones, hay múltiples opciones:
* Norma 2: $||f||_2 = \left( \int_a^bf(x)^2 dx \right)^\frac{1}{2}$
* Norma p: $||f||_p = \left( \int_a^b |f(x)|^p dx \right)^\frac{1}{p}$
* Norma inf: $||f||_{\inf} = \max_{x \in [a,b]} |f(x)|$

In [ ]:
f = x
g1 = sin(x)
g2 = 1 - cos(x)

In [ ]:
norma_p_func(f - g1, p=2, a=0, b=2*pi)

In [ ]:
norma_inf_func(f - g1, a=0, b=2*pi)

In [ ]:
norma_p_func(f - g2, p=2, a=0, b=2*pi)

In [ ]:
norma_inf_func(f - g2, a=0, b=2*pi)

### Aproximación por mínimos cuadrados (continua y discreta)

Para la aproximación definimos el producto escalar como una aplicación $\langle\,,\rangle \; :\,V \times V \rightarrow \mathbb{R}^+$ que verifica:
* $\langle f, f\rangle = \iff f = 0$
* $\langle f, g\rangle =\langle g, f\rangle$
* $\langle\alpha f+\mu g, h\rangle = \alpha\langle f, h\rangle + \mu \langle g, h\rangle$

La norma se define como $||f|| = \langle f, f\rangle^{\frac{1}{2}}$.

Una forma de producto escalar es, por ejemplo:
$$\langle f, g \rangle = \int_a^b f(x)g(x) dx$$

Esa forma puede aplicarse de forma discreta (para un conjunto discreto $I = \{x_1, x_2, \cdots, x_m\}$ como :
$$\langle f, g \rangle = \sum_{i \in I} f(i)g(i)$$ 

En la aproximación de mínimos cuadrados deseamos encontrar, para una base $\{\phi_0, \cdots, \phi_n\}$ de $U$, un vector $\alpha$ que minimice una función de coste $J(\alpha) = ||f - \sum_0^n \alpha_i\phi_i||^2$. Para ello se emplea el método de Gram, que consiste en encontrar la solución a 
$$G\alpha = \bar{f}$$
donde $G_{ij} = \langle\phi_i, \phi_j\rangle$ y $\bar{f}_i = \langle f, \phi_i\rangle$ en un producto escalar definido arbitrariamente.

In [ ]:
def producto_deriv(f, g, var, a=0, b=1, I=None):
    fg = f * g + f.diff(var) * g.diff(var)
    
    if I is None: # aplica el modo continuo
        integral = simplify(integrate(fg, (var, a, b)))
        return integral
    else:
        sum_I = S(0)  # Para hacerlo como objeto de sympy
        for i in I:
            sum_I += fg.subs(var, i)
        return simplify(sum_I)

def producto_asecas(f, g, var, a=0, b=1, I=None):
    fg = f * g
    if I is None: # aplica el modo continuo
        integral = simplify(integrate(fg, (var, a, b)))
        return integral
    else:
        sum_I = S(0)  # Para hacerlo como objeto de sympy
        for i in I:
            sum_I += fg.subs(var, i)
        return simplify(sum_I)

def metodo_gram(f, U, var, func_producto, a=0, b=1, I=None):
    G = zeros(len(U), len(U))
    f_bar = zeros(len(U), 1)
    
    for idx_i, u_i in enumerate(U):
        for idx_j, u_j in enumerate(U):
            if idx_i >= idx_j:
                g_ij = func_producto(u_i, u_j, var, a, b, I)
                G[idx_i, idx_j] = g_ij
                G[idx_j, idx_i] = g_ij
    
    for idx_i, u_i in enumerate(U):
        f_i = func_producto(f, u_i, var, a, b, I)
        f_bar[idx_i, 0] = f_i
    
    try:
        alpha = simplify((G ** -1) * f_bar)
    except:
        print('AVISO!!! La matriz G no es invertible. Aplicamos la pseudoinversa')
        from sympy.matrices.inverse import _pinv  # Aplica la pseudoinversa de Moore Penrose
        alpha = simplify(_pinv(G) * f_bar)
        
    expr_pol = simplify((alpha.T * mat([U]).T)[0]) # producto entre alpha y la base U
    
    return {'poly': expr_pol, 'alpha': alpha, 'f_bar': f_bar, 'G': G}

#### Ejercicio 33

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, a=0, b=1)

In [ ]:
dict_prod_deriv['poly']

In [ ]:
dict_prod_deriv['G']

In [ ]:
dict_prod_deriv['f_bar']

In [ ]:
dict_prod_deriv['alpha']

Ahora con una versión discreta en los puntos 0, 0.2, 0.4, 0.6, 0.8, 1:

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_deriv_discreto = metodo_gram(f, U, var=x, func_producto=producto_deriv, I = [0, 0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
dict_prod_deriv_discreto['poly']

In [ ]:
dict_prod_deriv_discreto['G']

In [ ]:
dict_prod_deriv_discreto['f_bar']

In [ ]:
dict_prod_deriv_discreto['alpha']

Ahora la versión continua pero con la función $\langle f, g \rangle = \int_a^b f(x)g(x) dx$

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_asecas = metodo_gram(f, U, var=x, func_producto=producto_asecas, a=0, b=1)

In [ ]:
dict_prod_asecas['poly']

In [ ]:
x_range = np.linspace(0, 1, 100)
y_real = [(E ** x).subs(x, i) for i in x_range]
y_prod_deriv = [dict_prod_deriv['poly'].subs(x, i) for i in x_range]
y_prod_discreto = [dict_prod_deriv_discreto['poly'].subs(x, i) for i in x_range]
y_prod_asecas = [dict_prod_asecas['poly'].subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label='real')
plt.plot(x_range, y_prod_deriv, label="fg + f'g' continuo")
plt.plot(x_range, y_prod_discreto, label="fg + f'g' discreto")
plt.plot(x_range, y_prod_asecas, label="fg")
plt.legend()

#### Ejercicio 34
En el ejercicio piden usar la base $\{1, x\}$. Nosotros lo hacemos con $\{1, x, x^2, x^3, x^4, x^5\}$, que da una representación más real de la aproximación.

Calculamos la versión continua

In [ ]:
U = [S(1), x, x ** 2, x ** 3, x ** 4]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = cos(pi * x)/2 + sin(pi/2 * x)/3
dict_prod_continuo = metodo_gram(f, U, var=x, func_producto=producto_asecas, a=-1, b=1)

In [ ]:
dict_prod_continuo['poly']

In [ ]:
dict_prod_continuo['G']

In [ ]:
dict_prod_continuo['f_bar']

In [ ]:
dict_prod_continuo['alpha']

Y ahora la discreta

In [ ]:
dict_prod_discreto = metodo_gram(f, U, var=x, func_producto=producto_asecas, I=[-1, 0, 1])

In [ ]:
dict_prod_discreto['poly']

In [ ]:
dict_prod_discreto['G']

In [ ]:
dict_prod_discreto['f_bar']

In [ ]:
dict_prod_discreto['alpha']

Repetimos el proceso, por curiosidad, pero con el producto escalar fg + f'g'

In [ ]:
dict_prod_continuo_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, a=-1, b=1)
dict_prod_discreto_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, I=[-1, 0, 1])

In [ ]:
dict_prod_continuo_deriv['poly']

In [ ]:
dict_prod_discreto_deriv['poly']

In [ ]:
x_range = np.linspace(-1, 1, 100)
y_real = [f.subs(x, i) for i in x_range]
y_prod_continuo = [dict_prod_continuo['poly'].subs(x, i) for i in x_range]
y_prod_discreto = [dict_prod_discreto['poly'].subs(x, i) for i in x_range]
y_prod_continuo_deriv = [dict_prod_continuo_deriv['poly'].subs(x, i) for i in x_range]
y_prod_discreto_deriv = [dict_prod_discreto_deriv['poly'].subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label='real')
plt.plot(x_range, y_prod_continuo, label="fg continuo")
plt.plot(x_range, y_prod_discreto, label="fg discreto")
plt.plot(x_range, y_prod_continuo_deriv, label="fg + f'g' continuo")
plt.plot(x_range, y_prod_discreto_deriv, label="fg + f'g' discreto")

plt.legend()

In [ ]:
def prod_esc_GS_cont(f, g, var, a=-1, b=1):
    return simplify(integrate(f * g, (var, a, b)))

def prod_esc_GS_disc(f, g, var, I=[-1, 1]):
    sum_I = S(0)
    for i in I:
        sum_I += f.subs(var, i) * g.subs(var, i)
        
    return simplify(sum_I)

def gram_schmidt_f(base, var, prod_esc, *args, **kwargs):
    list_pols_GS = []
    for i in range(len(base)):
        poli = base[i]
        for j in range(i):
            pj = list_pols_GS[j]
            poli -= (prod_esc(pj, base[i], var, *args, **kwargs))/(prod_esc(pj, pj, var, *args, **kwargs)) * pj
        
        list_pols_GS.append(poli)
    return list_pols_GS
    

#### Ejemplo 23

In [ ]:
gram_schmidt_f([S(1), x, x ** 2, x ** 3, x ** 4, x ** 5, x ** 6], x, prod_esc=prod_esc_GS_cont)

#### Ejercicio 35

In [ ]:
I = [-1, 0, 1, 2]
U_GS = gram_schmidt_f([S(1), x, x ** 2], x, prod_esc=prod_esc_GS_disc, I=I)

In [ ]:
for i in U_GS:
    print(i)

In [ ]:
f = sin(pi/2 * x)
dict_prod_discreto = metodo_gram(f, U_GS, var=x, func_producto=producto_asecas, I=I)

In [ ]:
dict_prod_discreto['poly']

### Aproximación con peso

Para las anteriores definiciones podemos tomar un producto escalar
$$\langle f, g, \rangle = \int_a^b f(x)g(x)\omega(x) dx$$

Donde $\omega(x)$ es una función positiva en $(a,b)$

Se puede construir la sucesión de polinomios
$$p_0(x) = 1$$
$$p_1(x) = x - a_1$$
$$p_2(x) = (x - a_2)p_1(x) - b_2p_0(x)$$
$$\cdots$$
$$p_n(x) = (x - a_n)p_{n-1}(x) - b_np_{n-2}(x)$$

Con
$$a_n = \frac{\langle xp_{n-1},p_{n-1}\rangle}{\langle p_{n-1},p_{n-1}\rangle}$$
$$b_n = \frac{\langle xp_{n-1},p_{n-2}\rangle}{\langle p_{n-2},p_{n-2}\rangle}$$

In [ ]:
def producto_escalar_peso(f, g, var, w=S(1),a=0, b=1, I=None):  # TODO - CORREGIR FUNCIONES
    fg = simplify(f * g * w)
    if I is None: # aplica el modo continuo
        integral = simplify(integrate(fg, (var, a, b)))
        return integral
    else:
        sum_I = S(0)  # Para hacerlo como objeto de sympy
        for i in I:
            sum_I += fg.subs(var, i)
        return simplify(sum_I)

    
def polinomios_orto_peso(base, w=S(1), var=x, a=-1, b=1, I=None):
    list_p, list_a, list_b = [], [], []
    
    # n = 0
    list_p.append(S(1))
    list_a.append(S(0))
    list_b.append(S(0))
    
    # n = 1
    a_i = simplify(producto_escalar_peso(base[0] * var, base[0], var=var, w=w, a=a, b=b, I=None) / 
                   producto_escalar_peso(base[0], base[0], var=var, w=w, a=a, b=b, I=None))
    list_p.append(var - a_i)
    list_a.append(a_i)
    list_b.append(S(0))
    
    # n > 1
    for i in range(2, len(base)):
        a_i = simplify(producto_escalar_peso(base[i - 1] * var, base[i - 1], var=var, w=w, a=a, b=b, I=None) / 
                       producto_escalar_peso(base[i - 1], base[i - 1], var=var, w=w, a=a, b=b, I=None))
        
        b_i = simplify(producto_escalar_peso(base[i - 1] * var, base[i - 2], var=var, w=w, a=a, b=b, I=None) / 
                       producto_escalar_peso(base[i - 2], base[i - 2], var=var, w=w, a=a, b=b, I=None))
        
        list_a.append(a_i)
        list_b.append(b_i)
        list_p.append(simplify((var - a_i) * list_p[i - 1] - b_i * list_p[i - 2]))
        
    return {'p': list_p, 'a': list_a, 'b': list_b}
        

#### Polinomios de Legendre

In [ ]:
polinomios_orto_peso(base_ortogonal)

In [ ]:
# La base para los polinomios tiene que ser ortogonal porque si no los polinomios no son ortogonales entre si!
base = [S(1), x, x**2, x**3, x**4, x**5, x**6]
base_ortogonal = gram_schmidt_f(base, var=x, prod_esc=prod_esc_GS_cont, a=-1, b=1)
base_legendre = polinomios_orto_peso(base_ortogonal, w=S(1), var=x, a=-1, b=1)

In [ ]:
base_legendre

In [ ]:
x_range = np.linspace(-1, 1, 100)

for p in base_legendre['p']:
    plt.plot(x_range, [p.subs(x, i)/abs(p.subs(x, -1)) for i in x_range], label=S(1)/abs(p.subs(x, -1)))
            
plt.legend(bbox_to_anchor=(1, 0.5))

In [ ]:
# Ahora probamos a resolver algun problema asociado. Por ejemplo, vamos a aproximar sin(x) en [0, 2 * pi]
base = [S(1), x, x**2, x**3, x**4]
base_ortogonal = gram_schmidt_f(base, var=x, prod_esc=prod_esc_GS_cont, a=0, b=2*pi)
base_legendre = polinomios_orto_peso(base_ortogonal, w=S(1), var=x, a=0, b=2*pi)

In [ ]:
metodo_gram

In [ ]:
f = sin(x)

U_GS_base = metodo_gram(f, base, var=x, func_producto=prod_esc_GS_cont, a=0, b=2*pi)
gram_schmidt_f(U, x, prod_esc=prod_esc_GS_cont, a=0, b=2*pi)

U_GS_ortogonal = gram_schmidt_f(base_ortogonal, x, prod_esc=prod_esc_GS_cont, a=0, b=2*pi)

U_GS_legendre = gram_schmidt_f(base_legendre['p'], x, prod_esc=prod_esc_GS_cont, a=0, b=2*pi)

In [ ]:
U_GS_base

In [ ]:
U_GS_legendre

In [ ]:
f = sin(pi/2 * x)
dict_prod_discreto = metodo_gram(f, U_GS, var=x, func_producto=producto_asecas, I=I)

In [ ]:
dict_prod_discreto['poly']

### Polinomios de Chebyshev